In [8]:
# 加载环境变量，如果你用的是OpenAI key就用这个方案！去掉注释即可！后面定义本地model的时候，把它注释
import dotenv
%load_ext dotenv
%dotenv

# test chat
from llama_index.llms.openai import OpenAI
response = OpenAI(model="gpt-3.5-turbo").complete("香蕉的颜色是")
print(response)

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv
黄色的。


In [9]:
import nest_asyncio
nest_asyncio.apply()

In [63]:
# 安装所有依赖
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple python-dotenv ipykernel llama-index nest_asyncio
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple timm
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple llama-index-llms-huggingface
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple llama-index-embeddings-huggingface
!pip install -i https://pypi.tuna.tsinghua.edu.cn/simple llama-index-embeddings-instructor

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [11]:
from llama_index.core import SimpleDirectoryReader

# 加载PDF数据，这里雄哥以卖油翁为例，可以改为自己的任意数据放在datasets文件夹即可，或改为指定的绝对路径
documents = SimpleDirectoryReader(input_files=["./datasets/maiyouweng.pdf"]).load_data()

In [27]:
from llama_index.core.node_parser import SentenceSplitter

# 把文本分块，chunk_size为1024，每一块的大小
splitter = SentenceSplitter(chunk_size=1024)
# 创建文档的节点
nodes = splitter.get_nodes_from_documents(documents)

In [39]:
# 这里可以打印每一个块的信息！
node_metadata = nodes[11].get_content(metadata_mode=True)
print(len(nodes), node_metadata)

12 page_label: 8
file_name: maiyouweng.pdf
file_path: datasets\maiyouweng.pdf
file_type: application/pdf
file_size: 429687
creation_date: 2024-06-18
last_modified_date: 2024-06-18

人物：陈尧咨和卖油翁。
背景：陈尧咨善射，“当世无双”，因而“自矜”。
9.汝亦知射乎？吾射不亦精乎”连用两个问句表现了陈尧咨怎样的性格特点？
先用一个疑问句，再用一个反问句，表现出陈尧咨因“善射”而“自矜”的性格特点。
10.卖油翁说“我亦无他，惟手熟尔”表明了什么？
表明自己并非如陈尧咨所认为的“轻吾射”，呼应了上文对陈尧咨射技的评价，再次说明“熟
能生巧”的道理；也表现了卖油翁不卑不亢、沉着镇静的性格特点。
11.“康肃笑而遣之”中的“笑”说明了什么？
说明陈尧咨看到卖油翁高超的技艺心服口服，认输了。
12.故事中只有两个人物，我们如果把题目《卖油翁》改成《陈尧咨和卖油翁》好吗?
不好。事端是卖油翁引起的，他如果不“释担而立”，不露出轻视的表情，陈尧咨就不会发问；
事情又是由他解决的——以酌油技术平息了陈尧咨的愤怒情绪。卖油翁的酌油表演，明显技
高一筹，是作者详写的部分；陈尧咨的射技，作者以“发矢十中八九”一笔带过，是略写。再
者，卖油翁身怀绝技，谦虚沉着；陈尧咨却因“善射”而自我炫耀，趾高气扬。卖油翁在事
件发展的全过程中起了主导作用。
13、高超的箭法真的能等同于往葫芦里灌油吗？好箭法真的只是“手熟”而已吗？再读课文，
把你的观点写下来。
从常识来看，从钱眼里注油入葫芦，不漏一滴，不沾钱孔，的确是手熟之技，但与高超的箭
发是不能等量齐观的。练成如此箭法不仅仅靠“手熟”两字，还要有过人的意志、超常的悟性、
良好的天赋等。正如有人说得好，“陈尧咨只要放下架子，肯定能学会倒油；卖油翁拼老命
也学不会拉弓搭箭，十中八九”，两者的技术含量不可同日而语。


In [4]:
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

C:\Users\ouhui\.conda\envs\xuehai3\lib\site-packages\pydantic\_internal\_fields.py:160: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


In [5]:
# 加载本地的qwen2-7b，你本地的模型放在哪就直接改路径
llm = HuggingFaceLLM(
    tokenizer_name="G:\\models\\Qwen2-7B-Instruct",
    model_name="G:\\models\\Qwen2-7B-Instruct",
    device_map="auto",
    tokenizer_kwargs={"trust_remote_code": True},
    model_kwargs={"trust_remote_code": True},
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [6]:
# 调用本地bce-embedding-base_v1作为embedding模型

embed_args = {
    'model_name': 'hkunlp/instructor-base', 
    'max_length': 512, 
    'embed_batch_size': 32, 
    'device': 'cuda'
    }
embed_model = HuggingFaceEmbedding(**embed_args)

C:\Users\ouhui\.conda\envs\xuehai3\lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [23]:
from llama_index.llms.openai import OpenAI
from llama_index.embeddings.openai import OpenAIEmbedding

llm = OpenAI(model="gpt-3.5-turbo")
embed_model = OpenAIEmbedding()

In [24]:
from llama_index.core import Settings
# 把加载的模型添加到环境设置中，定义全局模型

Settings.llm = llm
Settings.embed_model = embed_model

In [40]:
from llama_index.core import SummaryIndex, VectorStoreIndex

# 创建数据摘要索引
summary_index = SummaryIndex(nodes)
# 创建矢量存储索引
vector_index = VectorStoreIndex(nodes)

In [42]:
# 把这些索引，转换为工具，后面雄哥就可以调用这些工具
# 摘要查询引擎
summary_query_engine = summary_index.as_query_engine(
    response_mode="tree_summarize",
    use_async=True,
)

# 向量查询引擎
vector_query_engine = vector_index.as_query_engine()

In [43]:
from llama_index.core.tools import QueryEngineTool

# 定义summary_tool的工具
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    description=(
        "适用于生成与卖油翁课堂相关的摘要问题。"
    ),
)

# 定义vector_tool的工具
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    description=(
        "适用于检索卖油翁常规上下文的问题。"
    ),
)

In [44]:
from llama_index.core.query_engine.router_query_engine import RouterQueryEngine
from llama_index.core.selectors import LLMSingleSelector

# 定义所有的tools
query_engine = RouterQueryEngine(
    selector=LLMSingleSelector.from_defaults(),
    query_engine_tools=[
        summary_tool,
        vector_tool,
    ],
    verbose=True
)

In [45]:
import nest_asyncio
nest_asyncio.apply()

In [62]:
response = query_engine.query("卖油翁课堂有什么内容?")
print(str(response))

Selecting query engine 0: This choice is most relevant as it specifically mentions generating summary questions related to the content of the 卖油翁课堂..
在家里的射箭场地射箭，有个卖油的老翁放下担子，站在那里斜着眼睛看着他，很久都没有离开。


In [53]:
print(len(response.source_nodes))

2


In [61]:
response = query_engine.query("卖油翁的文章引发了什么思考？")
print(str(response))

Selecting query engine 1: This choice is more relevant as it pertains to retrieving general context about the article of 卖油翁..
The article "卖油翁" triggers contemplation about the importance of skill acquisition and the concept of "熟能生巧" (practice makes perfect). It also prompts reflection on the contrast between arrogance and humility, as depicted through the characters of 陈尧咨 and 卖油翁.


In [59]:
print(len(response.source_nodes))
for node in response.source_nodes:
    print(node)

2
Node ID: 52517623-2225-4901-9a76-4844ca60872f
Text: 5、读了课文，你认为一个人应该如何让看待自己和别人的长处？你想起哪些名言警句？ 三人行，必有我师焉。
择其善者而从之，其不善者而改之。 人外有人，天外有天。取人之长，补己之短。 谦虚使人进步，骄傲使人落后。 自满人十事九空，虚心人万事可成。
十一、问答题： 1.课文中的两个人物，哪一个是主要人物？为什么？
卖油翁。因为故事的目的是通过卖油翁对酌油的技巧的解释来揭示熟能生巧的道理。（意近 即可）
2.描述陈尧咨射箭和卖油翁酌油这两段文字，哪一段写得比较详细，哪一段写得比较简略， 这是为什么？
答：写卖油翁沥油一段详细，为突出他以理服暴，以“酌油”技艺制服对方的形象；略写陈尧
咨的射技，只用“十中八九”略写，对他恃技骄横则较详，便于突出各自性格，突出文章中心。 3.“有卖油翁释担而立，睨之...
Score:  0.853

Node ID: baefa59a-4aed-4cb4-8886-363cd3aef5c1
Text: 《卖油翁》 的创作背 景宋英宗治平 四年（1067 年）文学创作背景欧阳修因遭飞语中伤，自 请外任，在出知亳州时作《卖油翁》
《卖油翁》 中的主要 人物解读不明确人物形象分析卖油翁是主要人物，康肃 公是次要人物，通过陈尧咨的骄傲自满
突出了卖油翁的才艺和“业精于勤，熟 能生巧”的道理 《卖油翁》 的误读现 象 20世纪以来文学作品解读误区20世纪以来，《卖
油翁》被逐渐误读为讲述熟能生巧的道 理，而欧阳修的真实意图是通过故事揭 示射箭技能获得途径的重要性 《卖油翁》 的人物形
象分析不明确人物形象分析教师和学生对陈尧咨和 卖油翁的人物形象进行了不同的解读， 包括自以为是、傲慢自大的陈尧咨与谦
虚沉稳、深藏不露的卖油翁等观点 《卖油翁》 的故事寓 意不明确文学作品主题解读《卖油翁》通过陈...
Score:  0.850

